In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import random
import torch
import pandas as pd
from skimage import io
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
class gen_triplets(Dataset):
    def __init__(self,csv_file,root_dir,transform=None):
        self.df=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.transform=transform
        self.group2df=dict(list(self.df.groupby('label_group')))
        self.id_to_img=self.df.set_index('posting_id')['image'].to_dict()
        self.id_to_title=self.df.set_index('posting_id')["title"].to_dict()
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx=idx.tolist()
        anchor=self.df.loc[idx,"posting_id"]
        ids=self.group2df[self.df.loc[idx,"label_group"]]["posting_id"].tolist()
        ids.remove(anchor)
        positive=random.choice(ids)
        
        groups=list(self.group2df.keys())
        groups.remove(self.df.loc[idx,"label_group"])
        neg_group=random.choice(groups)
        negative=random.choice(self.group2df[neg_group]["posting_id"].to_list())
        print("Anchor: ",self.id_to_title[anchor],"\nPositive: ",self.id_to_title[positive],"\nNegative: ",self.id_to_title[negative])
        anchor_img=io.imread(os.path.join(self.root_dir,self.id_to_img[anchor]))
        positive_img=io.imread(os.path.join(self.root_dir,self.id_to_img[positive]))
        negative_img=io.imread(os.path.join(self.root_dir,self.id_to_img[negative]))
        if self.transform:
            anchor_img=transform(anchor_img)
            positive_img=transform(positive_img)
            negative_img=transform(negative_img)
                                                           
        
        return anchor_img,positive_img,negative_img
        

In [ ]:
transform=transforms.Compose([
     transforms.ToTensor(),
     transforms.Resize((128,128)),
])

In [ ]:
triples=gen_triplets("/kaggle/input/shopee-product-matching/train.csv","/kaggle/input/shopee-product-matching/train_images",transform=transform)

In [ ]:
anchor_img,positive_img,negative_img=triples[12]

In [ ]:
plt.imshow(anchor_img.permute(1,2,0))

In [ ]:
plt.imshow(positive_img.permute(1,2,0))

In [ ]:
plt.imshow(negative_img.permute(1,2,0))